In [18]:
# ✅ Retrieve stored variables from previous notebooks
%store -r dev_feature_store_table
%store -r prod_feature_store_table
%store -r dev_feature_group_name
%store -r prod_feature_group_name

# ✅ Verify the variables exist
if "dev_feature_store_table" not in locals() or "prod_feature_store_table" not in locals():
    raise ValueError("❌ Missing feature store table names. Run the setup notebook first!")

print("✅ Successfully loaded stored variables.")


no stored variable or alias dev_feature_store_table
no stored variable or alias prod_feature_store_table
no stored variable or alias dev_feature_group_name
no stored variable or alias prod_feature_group_name
✅ Successfully loaded stored variables.


In [19]:
import os
import boto3
import sagemaker
from sagemaker.feature_store.feature_group import FeatureGroup
from botocore.exceptions import ClientError
from pyathena import connect
import time

# ✅ Restore variables stored from previous notebooks
%store -r dev_feature_store_table
%store -r prod_feature_store_table
%store -r dev_feature_group_name
%store -r prod_feature_group_name

# ✅ Verify that all required variables are loaded
if not all(var in locals() for var in ["dev_feature_store_table", "prod_feature_store_table", "dev_feature_group_name", "prod_feature_group_name"]):
    raise ValueError("❌ Required variables are missing. Ensure the previous notebook has been run!")

# ✅ Initialize AWS Session
session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()

# ✅ Use SageMaker's default bucket
bucket = sagemaker_session.default_bucket()

# ✅ Set up Athena connection
s3_staging_dir = f's3://{bucket}/athena-query-results/'
conn = connect(s3_staging_dir=s3_staging_dir, region_name=region)

# ✅ Define variables
ATHENA_DATABASE = "sagemaker_featurestore"
ATHENA_TABLES_TO_DROP = [dev_feature_store_table, prod_feature_store_table]  # ✅ Fixed: Now uses stored table names
GLUE_DATABASE_TO_DROP = "db_airline_delay_cause"

# ✅ Initialize AWS clients
s3_client = boto3.client("s3")
athena_client = boto3.client("athena")
sagemaker_client = boto3.client("sagemaker", region_name=region)
glue_client = boto3.client("glue", region_name=region)

# ✅ Feature Group Names (Restored from %store)
DEV_FEATURE_GROUP_NAME = dev_feature_group_name
PROD_FEATURE_GROUP_NAME = prod_feature_group_name

# ✅ Delete both Dev & Prod Feature Groups
def delete_feature_groups():
    for feature_group_name in [DEV_FEATURE_GROUP_NAME, PROD_FEATURE_GROUP_NAME]:
        try:
            print(f"🔍 Checking if Feature Group `{feature_group_name}` exists...")
            existing_groups = sagemaker_client.list_feature_groups()['FeatureGroupSummaries']
            existing_group_names = [fg['FeatureGroupName'] for fg in existing_groups]

            if feature_group_name in existing_group_names:
                print(f"🚀 Feature Group `{feature_group_name}` found. Deleting...")
                sagemaker_client.delete_feature_group(FeatureGroupName=feature_group_name)

                # Wait until deletion is complete
                while True:
                    existing_groups = sagemaker_client.list_feature_groups()['FeatureGroupSummaries']
                    existing_group_names = [fg['FeatureGroupName'] for fg in existing_groups]
                    if feature_group_name not in existing_group_names:
                        print(f"✅ Feature Group `{feature_group_name}` deleted successfully.")
                        break
                    print("⏳ Waiting for Feature Group deletion...")
                    time.sleep(5)
            else:
                print(f"✅ Feature Group `{feature_group_name}` does not exist. No deletion needed.")
        except Exception as e:
            print(f"❌ Error deleting Feature Group `{feature_group_name}`: {e}")

# ✅ Run cleanup
def clean_state():
    print("\n🚀 **Starting Full Cleanup...**\n")
    delete_feature_groups()  # ✅ Now deletes both feature groups
    print("\n✅ **Cleanup completed successfully!**")

# ✅ Execute the cleanup function
clean_state()


no stored variable or alias dev_feature_store_table
no stored variable or alias prod_feature_store_table
no stored variable or alias dev_feature_group_name
no stored variable or alias prod_feature_group_name


ValueError: ❌ Required variables are missing. Ensure the previous notebook has been run!

# Use code below to check what else is on your system and whether something was left behind

In [22]:
import boto3
from botocore.exceptions import ClientError
from pyathena import connect

# ✅ Initialize AWS Session
session = boto3.session.Session()
region = session.region_name
sagemaker_session = boto3.Session()

# ✅ Use SageMaker's default bucket
bucket = sagemaker_session.client("s3").list_buckets()["Buckets"][0]["Name"]

# ✅ Set up Athena connection
s3_staging_dir = f's3://{bucket}/athena-query-results/'
conn = connect(s3_staging_dir=s3_staging_dir, region_name=region)

# ✅ Initialize AWS clients
s3_client = boto3.client("s3", region_name=region)
athena_client = boto3.client("athena", region_name=region)
sagemaker_client = boto3.client("sagemaker", region_name=region)
glue_client = boto3.client("glue", region_name=region)

# ✅ Function to list AWS resources
def list_aws_resources():
    print("\n--- 📌 AWS Resource Overview ---")
    
    # ✅ List Athena Databases
    try:
        databases = athena_client.list_databases(CatalogName="AwsDataCatalog")["DatabaseList"]
        database_names = [db["Name"] for db in databases]
        print("\n📌 **Athena Databases:**")
        for db in database_names:
            print(f"   - {db}")
    except ClientError as e:
        print("❌ Error listing Athena databases:", e)
        database_names = []  # Ensure it doesn't break the next step

    # ✅ List Athena Tables Per Database
    for database in database_names:
        try:
            tables = athena_client.list_table_metadata(CatalogName="AwsDataCatalog", DatabaseName=database)["TableMetadataList"]
            table_names = [table["Name"] for table in tables]

            print(f"\n📌 **Tables in Athena Database: `{database}`**")
            if table_names:
                for table in table_names:
                    print(f"   - {table}")
            else:
                print("   ❌ No tables found in this database.")

        except ClientError as e:
            print(f"❌ Error listing tables in `{database}`:", e)

    # ✅ List Feature Store Groups
    try:
        feature_groups = sagemaker_client.list_feature_groups()["FeatureGroupSummaries"]
        feature_group_names = [fg["FeatureGroupName"] for fg in feature_groups]
        print("\n📌 **Feature Store Groups:**")
        if feature_group_names:
            for fg in feature_group_names:
                print(f"   - {fg}")
        else:
            print("   ❌ No Feature Groups found.")
    except ClientError as e:
        print("❌ Error listing Feature Groups:", e)

    # ✅ List S3 Files
    try:
        objects = s3_client.list_objects_v2(Bucket=bucket)
        s3_files = [obj["Key"] for obj in objects.get("Contents", [])]
        print("\n📌 **S3 Files in Bucket `{bucket}`:**")
        if s3_files:
            for file in s3_files[:10]:  # Show only first 10 files for readability
                print(f"   - {file}")
            if len(s3_files) > 10:
                print(f"   ... ({len(s3_files)} total files)")
        else:
            print("   ❌ No files found.")
    except ClientError as e:
        print("❌ Error listing S3 files:", e)

    # ✅ List Glue Databases
    try:
        glue_databases = glue_client.get_databases()["DatabaseList"]
        glue_db_names = [db["Name"] for db in glue_databases]
        print("\n📌 **Glue Databases:**")
        if glue_db_names:
            for db in glue_db_names:
                print(f"   - {db}")
        else:
            print("   ❌ No Glue Databases found.")
    except ClientError as e:
        print("❌ Error listing Glue databases:", e)

# ✅ Run AWS resource listing
list_aws_resources()



--- 📌 AWS Resource Overview ---

📌 **Athena Databases:**
   - default

📌 **Tables in Athena Database: `default`**
   ❌ No tables found in this database.

📌 **Feature Store Groups:**
   ❌ No Feature Groups found.

📌 **S3 Files in Bucket `{bucket}`:**
   ❌ No files found.

📌 **Glue Databases:**
   - default
